In [1]:
import pandas as pd
import numpy as np
from sklearn import datasets
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from ReliefF import ReliefF
from sklearn.metrics import f1_score
from sklearn.neighbors import KNeighborsClassifier

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.lines as mlines

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from scipy import stats
from scipy import linalg
from scipy.stats import skew
from scipy.stats import kurtosis
from scipy.stats import entropy
import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.stats.multicomp as multi
from scipy.stats import levene
from scipy.stats import shapiro
from scipy.integrate import cumtrapz
from scipy.linalg import eig
from numpy.fft import fft
from ReliefF import ReliefF

In [2]:
def data_splitting_TT(X,y,test_size):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= test_size, random_state=11)

    return X_train, X_test, y_train, y_test

In [3]:
def data_splitting_TVT(X,y,validation_size,test_size):
    X_train1, X_val, y_train1, y_val = train_test_split(X, y, test_size = validation_size, random_state=11)
    X_train, X_test, y_train, y_test = train_test_split(X_train1, y_train1, test_size = test_size/(1-validation_size), random_state=11)


    return X_train, X_val, X_test, y_train, y_val, y_test

In [4]:
def data_splitting_K_fold(CV):
    kf = KFold(n_splits=CV)
    
    return kf

In [1]:
def metricas_exactidao(y_test,predictions):
    #Relatório de classificação
    #Ver como o modelo se ajusta para os dados de teste
    #print(classification_report(y_test, predictions))
    report = classification_report(y_test, predictions,output_dict=True)
    df=pd.DataFrame(report).transpose()
    f1 = df['f1-score'][-1]
    precision = df['precision'][-1]
    recall = df['recall'][-1]
    
    confusion = confusion_matrix(y_test,predictions)
    #Matriz de confusão
    #outside_columns = ["","Predicted",""]
    #outside_index = ["","Actual",""]
    #inside = ["Iris Setosa","Iris Versicolour","Iris Virginica"]
    
    #hier_columns = list(zip(outside_columns, inside))
    #hier_columns = pd.MultiIndex.from_tuples(hier_columns)
    
    #hier_index = list(zip(outside_index, inside))
    #hier_index = pd.MultiIndex.from_tuples(hier_index)
    
    
    return confusion, f1, precision, recall

In [6]:
def load_dataset():
    iris = datasets.load_iris()
    df = pd.DataFrame(data= np.c_[iris['data'], iris['target']],columns= iris['feature_names'] + ['target'])
    df.head()
    
    # X - variáveis usadas para modelar o modelo
    X = df.drop(["target"], axis=1)
    # y - variavel que se pretende avaliar
    y = df["target"]
    
    #sns.pairplot(df,hue="target",diag_kind='hist')
    #sns.FacetGrid(df,hue="target",size=5).map(sns.distplot,"petal length (cm)").add_legend()
    #sns.FacetGrid(df,hue="target",size=5).map(sns.distplot,"petal width (cm)").add_legend()
    #sns.FacetGrid(df,hue="target",size=5).map(sns.distplot,"sepal length (cm)").add_legend()
    #sns.FacetGrid(df,hue="target",size=5).map(sns.distplot,"sepal width (cm)").add_legend()
    #plt.show()
    
    return X,y

In [7]:
def select_features_ReliefF(X,y,numberFeatures,n_neighbors=10):
    y_data = y.to_numpy()
    features_names = X.columns
    X_data = X.to_numpy()
    fs = ReliefF(n_neighbors=n_neighbors, n_features_to_keep=numberFeatures)
    X_selected = fs.fit_transform(X_data, y_data)
    X_selected_name = pd.DataFrame()
    for j in range(0,numberFeatures):
        found = False
        i = 0
        while found == False:
            compare = (X_selected[:,j] == X_data[:,i])
            if sum(compare) == len(X_data[:,i]):
                found = True
            i += 1
        X_selected_name[str(features_names[i-1])] = X_selected[:,j]
        
    
    return X_selected_name

In [8]:
def select_features_ReliefF_human(X,y,numberFeatures,n_neighbors=10):
    y_data = y.to_numpy()
    features_names = X.columns
    X_data = X.to_numpy()
    fs = ReliefF(n_neighbors=n_neighbors, n_features_to_keep=numberFeatures)
    X_selected = fs.fit_transform(X_data, y_data)
    X_selected_name = pd.DataFrame()
    _,selected_features = np.shape(X_selected)
    found_column = [0]
    for j in range(0,selected_features):
        found = False
        i = 0
        while found == False:
            compare = (X_selected[:,j] == X_data[:,i])
            if (sum(compare) == len(X_data[:,i])) & (i not in found_column):
                found = True
                found_column.append(i)
            i += 1
        X_selected_name[str(features_names[i-1])] = X_selected[:,j]       
    
    return X_selected_name

In [9]:
def evaluate_metric(model, x_test, y_test):
    return f1_score(y_test, model.predict(x_test), average='micro')

In [10]:
# https://towardsdatascience.com/feature-importance-and-forward-feature-selection-752638849962

def forward_feature_selection(X_train, X_test, y_train, y_test,num_features,n_neighbors,distance,weights):
    feature_set = []
    f1_set = []    
    for num_features in range(num_features):
        metric_list = [] # Choose appropriate metric based on business problem
        model = KNeighborsClassifier(n_neighbors=n_neighbors,metric=distance,weights=weights) # You can choose any model you like, this technique is model agnostic
        for feature in X_train.columns:
            if feature not in feature_set:
                f_set = feature_set.copy()
                f_set.append(feature)
                model.fit(X_train[f_set], y_train)
                metric_list.append((evaluate_metric(model, X_test[f_set], y_test), feature))

        metric_list.sort(key=lambda x : x[0], reverse = True) # In case metric follows "the more, the merrier"
        feature_set.append(metric_list[0][1])
        f1_set.append(metric_list[0][0])

    #ffs = {feature_set[0]:f1_set[0],feature_set[1]:f1_set[1],feature_set[2]:f1_set[2],feature_set[3]:f1_set[3]}
    return feature_set,f1_set

In [11]:
# ex 2.4
def dataset_reduced():
    X, y = load_dataset()
    X["target"] = y
    X_0 = X[X["target"]==0]
    X_1 = X[X["target"]==1]
    X_1 = X_1.sample(30)
    X_2 = X[X["target"]==2]
    X_2 = X_2.sample(10)
    X_reduced = X_0
    X_reduced = X_reduced.append(X_1)
    X_reduced = X_reduced.append(X_2)
    X_reduced = X_reduced.sample(frac = 1)
    X_reduced = X_reduced.reset_index(drop=True)
    y_reduced = X_reduced["target"]
    X_reduced = X_reduced.drop("target",axis=1)
    return X_reduced, y_reduced 

In [12]:
# excercise 3

def dataset(patient_id):
    head=["device_id","acc_x","acc_y","acc_z","gyro_x","gyro_y","gyro_z","mag_x","mag_y","mag_z","time","activity","patient_id"]

    #device_1 = pd.read_csv("dataset/part{patient}/part{patient}dev1.csv".format(patient=patient_id),names=head)
    device_2 = pd.read_csv("dataset/part{patient}/part{patient}dev2.csv".format(patient=patient_id),names=head)
    #device_3 = pd.read_csv("dataset/part{patient}/part{patient}dev3.csv".format(patient=patient_id),names=head)
    #device_4 = pd.read_csv("dataset/part{patient}/part{patient}dev4.csv".format(patient=patient_id),names=head)
    #device_5 = pd.read_csv("dataset/part{patient}/part{patient}dev5.csv".format(patient=patient_id),names=head)
    
    
    #df=pd.concat([device_1,device_2,device_3,device_4,device_5],axis=0)
    df=device_2
    df["patient_id"]=patient_id
    return df

In [13]:
# excercise 3

def transformed_dataset():

    df_transformed = pd.DataFrame()

    for i in range (0,15):
        df_transformed = df_transformed.append(dataset(i))
    df_transformed = df_transformed.reset_index()
        
    df_transformed["acc_vector"] = df_transformed["acc_x"]**2+df_transformed["acc_y"]**2+df_transformed["acc_z"]**2
    df_transformed["gyro_vector"] = df_transformed["gyro_x"]**2+df_transformed["gyro_y"]**2+df_transformed["gyro_z"]**2
    df_transformed["mag_vector"] = df_transformed["mag_x"]**2+df_transformed["mag_y"]**2+df_transformed["mag_z"]**2
    
    return df_transformed[["device_id","acc_vector","gyro_vector","mag_vector","time","activity","patient_id"]]

In [14]:
# exercice 4.2

def dataset_act_vector(act,vector):    
    
    df = pd.DataFrame()
    vector_x = vector+'_x'
    vector_y = vector+'_y'
    vector_z = vector+'_z'

    for i in range (0,15):
        df = df.append(dataset(i))
    df = df.reset_index()
    df_act_vector = df[df["activity"]==act][[vector_x,vector_y,vector_z,'time']].reset_index(drop=True)
    df_act_vector = df_act_vector.sort_values(by=["time"]).reset_index(drop=True)
    
    return df_act_vector

In [15]:
# exercice 4.2

def statistical_features(initial,final,act,vector):
    
    df_act_vector = dataset_act_vector(act,vector)
    df_window = df_act_vector[(df_act_vector['time']>=initial) & (df_act_vector['time']<=final)].reset_index(drop=True)
    results = df_window.describe()
    variance = np.sqrt(results.loc['std']) 
    results.loc['variance'] = variance
    root_mean_square = np.sqrt(np.sum(df_window**2)/df_window.count())
    results.loc['rms'] = root_mean_square
    vector_x = vector+'_x'
    vector_y = vector+'_y'
    vector_z = vector+'_z'
    derivative = np.zeros((4))
    derivative[0] = np.mean(np.gradient(df_window[vector_x],df_window['time']))
    derivative[1] = np.mean(np.gradient(df_window[vector_y],df_window['time']))
    derivative[2] = np.mean(np.gradient(df_window[vector_z],df_window['time']))
    results.loc['av_der'] = derivative
    skewness = np.zeros(4)
    skewness[0] = skew(df_window[vector_x])
    skewness[1] = skew(df_window[vector_y])
    skewness[2] = skew(df_window[vector_z])
    results.loc['skew'] = skewness
    kurt = np.zeros(4)
    kurt[0] = kurtosis(df_window[vector_x])
    kurt[1] = kurtosis(df_window[vector_y])
    kurt[2] = kurtosis(df_window[vector_z])
    results.loc['kurtosis'] = kurt
    interq_range = results.loc['75%']-results.loc['25%']
    results.loc['interq_range'] = interq_range
    zero_crossings = np.zeros(4)
    #https://stackoverflow.com/questions/3843017/efficiently-detect-sign-changes-in-python
    zero_crossings[0] = len(np.where(np.diff(np.sign(df_window[vector_x])))[0])/(final-initial)
    zero_crossings[1] = len(np.where(np.diff(np.sign(df_window[vector_y])))[0])/(final-initial)
    zero_crossings[2] = len(np.where(np.diff(np.sign(df_window[vector_z])))[0])/(final-initial)
    results.loc['zero_crossings'] = zero_crossings
    mean_crossings = np.zeros(4)
    #https://stackoverflow.com/questions/3843017/efficiently-detect-sign-changes-in-python
    mean_crossings[0] = len(np.where(np.diff(np.sign(df_window[vector_x]-results.loc['mean'][vector_x])))[0])/(final-initial)
    mean_crossings[1] = len(np.where(np.diff(np.sign(df_window[vector_y]-results.loc['mean'][vector_y])))[0])/(final-initial)
    mean_crossings[2] = len(np.where(np.diff(np.sign(df_window[vector_z]-results.loc['mean'][vector_z])))[0])/(final-initial)
    results.loc['mean_crossings'] = mean_crossings
    pair_correlation = df_window.corr()
    results.loc['corr_'+vector_x] = pair_correlation.loc[vector_x]
    results.loc['corr_'+vector_y] = pair_correlation.loc[vector_y]
    results.loc['corr_'+vector_z] = pair_correlation.loc[vector_z]
    spec_entropy = np.zeros(4)
    p_data = df_window[vector_x].value_counts()
    spec_entropy[0] = entropy(p_data)
    p_data = df_window[vector_y].value_counts()
    spec_entropy[1] = entropy(p_data)
    p_data = df_window[vector_z].value_counts()
    spec_entropy[2] = entropy(p_data)
    results.loc['spec_entropy'] = spec_entropy
    
    return results


In [16]:
# exercice 4.2

def moviment_intensity(initial,final,act): 
    vector="acc"
    df_act_vector = dataset_act_vector(act,vector)
    df_window = df_act_vector[(df_act_vector['time']>=initial) & (df_act_vector['time']<=final)].reset_index(drop=True)
    vector_x = "acc_x"
    vector_y = "acc_y"
    vector_z = "acc_z"
    df_window = df_window[[vector_x,vector_y,vector_z]]
    MI = np.sqrt(np.sum(df_window**2,axis=1))
    AI = np.sum(MI)/(final-initial)
    VI = np.sum((MI-AI)**2)/(final-initial)
    
    return AI,VI

In [17]:
# exercice 4.2

def normalized_signal_magnitude_area(initial,final,act):
    vector="acc"
    df_act_vector = dataset_act_vector(act,vector)
    df_window = df_act_vector[(df_act_vector['time']>=initial) & (df_act_vector['time']<=final)].reset_index(drop=True)
    vector_x = "acc_x"
    vector_y = "acc_y"
    vector_z = "acc_z"
    df_window = df_window[[vector_x,vector_y,vector_z]]
    SMA = np.sum(np.sum(abs(df_window)))/(final-initial)
    
    return SMA

In [18]:
# exercice 4.2

def eigen_values_dominant_directions(initial,final,act):
    vector="acc"
    df_act_vector = dataset_act_vector(act,vector)
    df_window = df_act_vector[(df_act_vector['time']>=initial) & (df_act_vector['time']<=final)].reset_index(drop=True)
    vector_x = "acc_x"
    vector_y = "acc_y"
    vector_z = "acc_z"
    df_window = df_window[[vector_x,vector_y,vector_z]]
    covariance_matrix = np.cov(df_window)
    eigvals,eigvecs = eig(covariance_matrix)
    eigvals = np.array(sorted(eigvals, reverse=True)[:2])
    
    return eigvals

In [19]:
def correlation_acceleration_gravity_heading_directions(initial,final,act):
    vector="acc"
    df_act_vector = dataset_act_vector(act,vector)
    df_window = df_act_vector[(df_act_vector['time']>=initial) & (df_act_vector['time']<=final)].reset_index(drop=True)
    vector_x = "acc_x"
    vector_y = "acc_y"
    vector_z = "acc_z"
    df_window = df_window[[vector_x,vector_y,vector_z]]
    gravity = df_window[vector_x]
    heading = df_window[[vector_y,vector_z]]
    heading_norm = np.sqrt(np.sum(heading**2,axis=1))
    df_gravity_heading = pd.DataFrame({'gravity':gravity,'heading_norm':heading_norm})
    correlation = df_gravity_heading.corr()
    correlation = correlation["heading_norm"][0]
    
    return correlation

In [20]:
def averaged_velocity_heading_direction(initial,final,act):
    vector="acc"
    df_act_vector = dataset_act_vector(act,vector)
    df_window = df_act_vector[(df_act_vector['time']>=initial) & (df_act_vector['time']<=final)].reset_index(drop=True)
    vector_y = vector+'_y'
    vector_z = vector+'_z'
    velocity_y = np.sum(cumtrapz(df_window[vector_y],df_window['time']))/(final-initial)
    velocity_z = np.sum(cumtrapz(df_window[vector_z],df_window['time']))/(final-initial)
    norm_velocity = np.sqrt(velocity_y**2 + velocity_z**2)
    
    return norm_velocity

In [21]:
# exercice 4.2

def averaged_velocity_gravity_direction(initial,final,act):
    vector = "acc"
    df_act_vector = dataset_act_vector(act,vector)
    df_window = df_act_vector[(df_act_vector['time']>=initial) & (df_act_vector['time']<=final)].reset_index(drop=True)
    vector_x = vector+'_x'
    velocity_x = np.sum(cumtrapz(df_window[vector_x],df_window['time']))/(final-initial)
    
    return velocity_x

In [22]:
# exercice 4.2

def averaged_rotation_angles_gravity_direction(initial,final,act):
    vector="gyro"
    df_act_vector = dataset_act_vector(act,vector)
    df_window = df_act_vector[(df_act_vector['time']>=initial) & (df_act_vector['time']<=final)].reset_index(drop=True)
    vector_x = vector+'_x'
    av_rotation = np.sum(df_window[vector_x])/(final-initial)
    
    return av_rotation

In [23]:
# exercice 4.2

def dominant_frequency(initial,final,act,vector):
    df_act_vector = dataset_act_vector(act,vector)
    df_window = df_act_vector[(df_act_vector['time']>=initial) & (df_act_vector['time']<=final)].reset_index(drop=True)
    vector_x = vector+'_x'
    vector_y = vector+'_y'
    vector_z = vector+'_z'
    df_window = df_window[[vector_x,vector_y,vector_z]]
    fourier = fft.fft(df_window)
    freq = fft.fftfreq(len(fourier))
    index_max = np.unravel_index(abs(fourier).argmax(), fourier.shape)
    frequency = freq(index_max[0])
    
    return frequency

In [24]:
# exercice 4.2

def energy(initial,final,act,vector):
    df_act_vector = dataset_act_vector(act,vector)
    df_window = df_act_vector[(df_act_vector['time']>=initial) & (df_act_vector['time']<=final)].reset_index(drop=True)
    vector_x = vector+'_x'
    vector_y = vector+'_y'
    vector_z = vector+'_z'
    df_window = df_window[[vector_x,vector_y,vector_z]]
    dc = df_window - df_window.mean()
    fourier = np.fft.fft(dc)
    en_x,en_y,en_z = abs(fourier).sum(axis=0)/(final-initial)
    
    return en_x,en_y,en_z

In [25]:
# exercice 4.2

def averaged_acceleration_energy(initial,final,act):
    vector = 'acc'
    en_x,en_y,en_z = energy(initial,final,act,vector)
    aae = np.mean([en_x,en_y,en_z])
    
    return aae

In [26]:
# exercice 4.2

def averaged_rotation_energy(initial,final,act):
    vector = 'gyro'
    en_x,en_y,en_z = energy(initial,final,act,vector)
    are = np.mean([en_x,en_y,en_z])
    
    return are

In [27]:
# exercice 4.2

#window_size=40000
#act=1
#vector="acc"

def features_dataset_by_activity(window_size,act,vector):
    warnings.filterwarnings("ignore")
    
    data = pd.DataFrame(columns=[vector+"_x_mean",vector+"_y_mean",vector+"_z_mean",
                                 vector+"_x_median",vector+"_y_median",vector+"_z_median",
                                 vector+"_x_std",vector+"_y_std",vector+"_z_std",
                                 vector+"_x_variance",vector+"_y_variance",vector+"_z_variance",
                                 vector+"_x_rms",vector+"_y_rms",vector+"_z_rms",
                               #  vector+"_x_av_der",vector+"_y_av_der",vector+"_z_av_der",
                                 vector+"_x_skew",vector+"_y_skew",vector+"_z_skew",
                                 vector+"_x_kurtosis",vector+"_y_kurtosis",vector+"_z_kurtosis",
                                 vector+"_x_interq_range",vector+"_y_interq_range",vector+"_z_interq_range",
                                 vector+"_x_zero_crossings",vector+"_y_zero_crossings",vector+"_z_zero_crossings",
                                 vector+"_x_mean_crossings",vector+"_y_mean_crossings",vector+"_z_mean_crossings",
                                 vector+"_x_corr_acc_x",vector+"_y_corr_acc_x",vector+"_z_corr_acc_x",
                                 vector+"_x_corr_acc_y",vector+"_y_corr_acc_y",vector+"_z_corr_acc_y",
                                 vector+"_x_corr_acc_z",vector+"_y_corr_acc_z",vector+"_z_corr_acc_z",
                                 vector+"_x_spec_entropy",vector+"_y_spec_entropy",vector+"_z_spec_entropy",
                                 
                                 "acc_MI","acc_VI","acc_SMA","acc_gravity_heading_corr","acc_avg_vel_head", "acc_avg_vel_grav", 
                                 "gyro_avg_rot",vector+"_x_en",vector+"_y_en",vector+"_z_en","acc_aae","gyro_are"])

    df_win=pd.DataFrame(columns=["init","final"])

    df = dataset_act_vector(act,vector)

    for i in range(0,len(df)-window_size,int(window_size/2)):
        df_win.loc[i]=[df["time"][i],df["time"][i+window_size]]
    
    df_win=df_win.reset_index(drop=True)
    
    for i in range(0,len(df_win)):
        initial = df_win["init"][i]
        final = df_win["final"][i]
    
        results = statistical_features(initial,final,act,vector)
        ai,vi = moviment_intensity(initial,final,act)
        sma = normalized_signal_magnitude_area(initial,final,act)
        #eigvals = eigen_values_dominant_directions(initial,final,act)
        corr = correlation_acceleration_gravity_heading_directions(initial,final,act)
        avg_vel_head = averaged_velocity_heading_direction(initial,final,act)
        avg_vel_grav = averaged_velocity_gravity_direction(initial,final,act)
        avg_rot = averaged_rotation_angles_gravity_direction(initial,final,act)
        en_x,en_y,en_z = energy(initial,final,act,vector)
        aae = averaged_acceleration_energy(initial,final,act)
        are = averaged_rotation_energy(initial,final,act)
    
        data.loc[i]=[results.loc["mean"][0],results.loc["mean"][1],results.loc["mean"][2],
                     results.loc["50%"][0],results.loc["50%"][1],results.loc["50%"][2],
                     results.loc["std"][0],results.loc["std"][1],results.loc["std"][2],
                     results.loc["variance"][0],results.loc["variance"][1],results.loc["variance"][2],
                     results.loc["rms"][0],results.loc["rms"][1],results.loc["rms"][2],
                   #  results.loc["av_der"][0],results.loc["av_der"][1],results.loc["av_der"][2],
                     results.loc["skew"][0],results.loc["skew"][1],results.loc["skew"][2],
                     results.loc["kurtosis"][0],results.loc["kurtosis"][1],results.loc["kurtosis"][2],
                     results.loc["interq_range"][0],results.loc["interq_range"][1],results.loc["interq_range"][2],
                     results.loc["zero_crossings"][0],results.loc["zero_crossings"][1],results.loc["zero_crossings"][2],
                     results.loc["mean_crossings"][0],results.loc["mean_crossings"][1],results.loc["mean_crossings"][2],
                     results.loc["corr_acc_x"][0],results.loc["corr_acc_x"][1],results.loc["corr_acc_x"][2],
                     results.loc["corr_acc_y"][0],results.loc["corr_acc_y"][1],results.loc["corr_acc_y"][2],
                     results.loc["corr_acc_z"][0],results.loc["corr_acc_z"][1],results.loc["corr_acc_z"][2],
                     results.loc["spec_entropy"][0],results.loc["spec_entropy"][1],results.loc["spec_entropy"][2],
                     
                     ai,vi,sma,corr,avg_vel_head,avg_vel_grav,avg_rot,en_x,en_y,en_z,aae,are]
        
    return data

In [28]:
# exercice 4.2

def features_dataset_activities(window_size,vector):

    data_final = []
    for i in range (1,17):
        data = features_dataset_by_activity(window_size,i,vector)
        data["activity"] = i
        data_final.append(data)
    features_data = pd.concat(data_final).reset_index(drop=True)
    
    return features_data

In [3]:
def Fisher_Score(X_data,y_data):
    Score = np.zeros(X_data.shape[1])
    categories = np.unique(y_data)
    for f in range(0,X_data.shape[1]):
        data_feature = X_data.iloc[:,f]
        mean_feature = data_feature.mean()
        frequency_categories = np.zeros(len(categories))
        mean_categories = np.zeros(len(categories))
        var_categories = np.zeros(len(categories))
        for i in range(0,len(categories)-1):
            data_feature_category = data_feature[y_data==categories[i]]
            frequency_categories[i] = len(data_feature_category)
            mean_categories[i] = data_feature_category.mean()
            var_categories[i] = data_feature_category.var()
        Score[f] = sum(frequency_categories * ((mean_categories-mean_feature)**2)) / max(sum(frequency_categories * var_categories**2),10**(-5))
    return Score

In [4]:
def select_features_Fisher(features_data,numberFeatures):
    y_data = features_data["activity"]
    X_data = features_data.drop(columns = "activity")
    Score = Fisher_Score(X_data,y_data)
    Score[np.isnan(Score)] = 0
    idx = np.argsort(Score, 0)
    idx = idx[::-1]
    data_Score = pd.DataFrame()
    data_Score = X_data.iloc[:,idx[0:numberFeatures]]
    FisherScore_selected = pd.DataFrame([Score[idx[0:numberFeatures]]],columns=data_Score.columns)
    
    return data_Score, FisherScore_selected